# Movie Cleanup

## Imports

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',100)

## Load and View Data

In [2]:
basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url='https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [4]:
basics.info()
basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10246265 entries, 0 to 10246264
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 703.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
akas.info()
akas.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37517220 entries, 0 to 37517219
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.info()
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360168 entries, 0 to 1360167
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1360168 non-null  object 
 1   averageRating  1360168 non-null  float64
 2   numVotes       1360168 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.1+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1893
3,tt0000004,5.5,178
4,tt0000005,6.2,2678


## Basics

### Replace "\N" with np.nan

In [7]:
basics.replace({'\\N':np.nan}, inplace=True)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


### Eliminate movies that are null for runtimeMinutes and genre

In [8]:
basics.isna().sum()


tconst                   0
titleType                0
primaryTitle            11
originalTitle           11
isAdult                  1
startYear          1369767
endYear           10132437
runtimeMinutes     7156284
genres              458549
dtype: int64

In [9]:
basics.dropna(subset=['runtimeMinutes','genres'], inplace=True)

In [10]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          190840
endYear           2956527
runtimeMinutes          0
genres                  0
dtype: int64

### keep only titleType==Movie

In [11]:
basics['titleType'].value_counts()

tvEpisode       1584015
short            617079
movie            390734
video            185702
tvMovie           92905
tvSeries          92429
tvSpecial         19203
tvMiniSeries      18087
tvShort            8746
videoGame           339
Name: titleType, dtype: int64

In [12]:
basics.drop(basics[basics['titleType'] != 'movie'].index, inplace=True)

In [13]:
basics['titleType'].value_counts()

movie    390734
Name: titleType, dtype: int64

### keep startYear 2000-2022

In [14]:
basics.dropna(subset=['startYear'], inplace=True)

In [15]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [16]:
basics['startYear'].value_counts()

2018    14443
2017    14432
2019    14186
2016    14013
2015    13538
        ...  
1894        1
1899        1
1904        1
1897        1
2028        1
Name: startYear, Length: 132, dtype: int64

In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384090 entries, 8 to 10246215
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          384090 non-null  object
 1   titleType       384090 non-null  object
 2   primaryTitle    384090 non-null  object
 3   originalTitle   384090 non-null  object
 4   isAdult         384090 non-null  object
 5   startYear       384090 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  384090 non-null  object
 8   genres          384090 non-null  object
dtypes: object(9)
memory usage: 29.3+ MB


In [18]:
basics['startYear'] = basics['startYear'].astype(int)

In [19]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384090 entries, 8 to 10246215
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          384090 non-null  object
 1   titleType       384090 non-null  object
 2   primaryTitle    384090 non-null  object
 3   originalTitle   384090 non-null  object
 4   isAdult         384090 non-null  object
 5   startYear       384090 non-null  int64 
 6   endYear         0 non-null       object
 7   runtimeMinutes  384090 non-null  object
 8   genres          384090 non-null  object
dtypes: int64(1), object(8)
memory usage: 29.3+ MB


In [20]:
basics.drop(basics[basics['startYear'] < 2000].index, inplace=True)

In [21]:
basics.drop(basics[basics['startYear'] > 2022].index, inplace=True)

In [22]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13081,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61111,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67485,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67663,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


In [23]:
basics['startYear'].value_counts()

2018    14443
2017    14432
2019    14186
2016    14013
2015    13538
2022    13324
2014    13176
2021    12559
2013    12425
2012    11687
2020    11667
2011    10810
2010    10250
2009     9405
2008     8207
2007     7000
2006     6567
2005     5876
2004     5235
2003     4634
2002     4156
2001     3900
2000     3666
Name: startYear, dtype: int64

### Eliminate movies that include "Documentary" in genre

In [24]:
basics['genres'].value_counts()

Documentary                  53781
Drama                        36262
Comedy                       13489
Comedy,Drama                  6486
Horror                        5928
                             ...  
Adult,Mystery                    1
Biography,Music,Mystery          1
Action,Animation,History         1
Comedy,Reality-TV,Romance        1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 1172, dtype: int64

In [25]:
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [26]:
basics['genres'].value_counts()

Drama                        36262
Comedy                       13489
Comedy,Drama                  6486
Horror                        5928
Drama,Romance                 4346
                             ...  
Animation,Biography,Sport        1
Adventure,History,Music          1
Adventure,History,War            1
Adventure,Romance,Sport          1
Crime,Fantasy,Sci-Fi             1
Name: genres, Length: 954, dtype: int64

## Akas

### Replace "\N" with np.nan

In [27]:
akas.replace({'\\N':np.nan}, inplace=True)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


### keep only US movies.

In [28]:
akas['region'].value_counts()['US']

1472163

In [29]:
akas.drop(akas[akas['region'] != 'US'].index, inplace=True)

In [30]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


### US  Movie Filtering for "Basics"

In [31]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34800        True
61111        True
67485        True
67663        True
80548        True
            ...  
10245947     True
10245986    False
10246031     True
10246115    False
10246205    False
Name: tconst, Length: 148575, dtype: bool

In [32]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61111,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67485,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
67663,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
80548,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror
...,...,...,...,...,...,...,...,...,...
10245404,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
10245798,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019,NaN,97,"Comedy,Drama,Fantasy"
10245938,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
10245947,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


## ratings

In [33]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1893
3,tt0000004,5.5,178
4,tt0000005,6.2,2678


### Check for NaN Values

In [34]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

### US only Filtering for "ratings"

In [35]:
keepers2 = ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1360163    False
1360164    False
1360165    False
1360166    False
1360167    False
Name: tconst, Length: 1360168, dtype: bool

In [36]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
4,tt0000005,6.2,2678
5,tt0000006,5.0,182
6,tt0000007,5.4,838
...,...,...,...
1360130,tt9916200,8.1,238
1360131,tt9916204,8.2,274
1360138,tt9916348,8.3,18
1360139,tt9916362,6.4,5571


## Final .Info()

In [37]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87354 entries, 34800 to 10246031
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          87354 non-null  object
 1   titleType       87354 non-null  object
 2   primaryTitle    87354 non-null  object
 3   originalTitle   87354 non-null  object
 4   isAdult         87354 non-null  object
 5   startYear       87354 non-null  int64 
 6   endYear         0 non-null      object
 7   runtimeMinutes  87354 non-null  object
 8   genres          87354 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.7+ MB


In [39]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472163 entries, 5 to 37516964
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472163 non-null  object
 1   ordering         1472163 non-null  int64 
 2   title            1472163 non-null  object
 3   region           1472163 non-null  object
 4   language         4168 non-null     object
 5   types            984662 non-null   object
 6   attributes       47623 non-null    object
 7   isOriginalTitle  1470822 non-null  object
dtypes: int64(1), object(7)
memory usage: 101.1+ MB


In [40]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512001 entries, 0 to 1360144
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         512001 non-null  object 
 1   averageRating  512001 non-null  float64
 2   numVotes       512001 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.6+ MB


## Saving DataFrames

In [41]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [42]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [43]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

## Reading Saved dataframes

In [44]:
basics2 = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008,NaN,94,Horror


In [45]:
akas2 = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas2.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [46]:
ratings2 = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings2.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000005,6.2,2678
3,tt0000006,5.0,182
4,tt0000007,5.4,838
